In [ ]:
from bs4 import BeautifulSoup
test_html = '''
<html>
  <body>
    <h4>学号</h4>
    <ul>
       <li>2112001</li>
       <li>2112002</li>
       <li class='blank'>2112003</li>
       <li>2112004</li>
    </ul>
    <h4>姓名</h4>
    <ul class="ul" style="color:red">
        <li>张三</li>
        <li>李四</li>
        <li>王五</li>
        <li>老六</li>
    </ul>
  </body>
</html>
'''
soup = BeautifulSoup(test_html, 'html.parser')

In [5]:
from bs4 import BeautifulSoup
import requests
# 设置代理服务器
headers = {
    'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
#请求连接
url = "http://book.chenlove.cn/all/id/18.html"
response = requests.get(url, headers=headers)
if response.status_code == 200:
    # 转化为utf-8格式，不加这条语句，输出爬取的信息为乱码
    response.encoding = 'utf8'
    # 把网页解析为BeautifulSoup对象
    soup = BeautifulSoup(response.text, 'html.parser')
    for element in soup.find_all(['dl', ['dd']]):
        a = element.find('a')
        if a.string!=None:
            print(a.string)
            print(a.get("href"))

ConnectionError: HTTPConnectionPool(host='book.chenlove.cn', port=80): Max retries exceeded with url: /all/id/18.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10df79460>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [4]:
# -*- codeing = utf-8 -*-
from bs4 import BeautifulSoup           # 网页解析
import os.path                          # 文件操作
import urllib.request, urllib.error     # URL操作，获取网页数据
import xlwt                             # excel操作
 
 
# 获取原始 html 网页
def readHtml():
    print("—————————— Read ——————————")
    # 要爬取的网页链接
    baseurl = "https://movie.douban.com/chart"
    # 模拟浏览器头部信息，向豆瓣服务器发送消息
    # 用户代理，表示告诉豆瓣服务器，我们是什么类型的机器、浏览器（本质上是告诉浏览器，我们可以接收什么水平的文件内容）
    head = {
        "User-Agent": "Mozilla / 5.0(Windows NT 10.0; Win64; x64) AppleWebKit / 537.36(KHTML, like Gecko) Chrome / 80.0.3987.122  Safari / 537.36"
    }
    request = urllib.request.Request(baseurl, headers=head)
    response = urllib.request.urlopen(request)
    html = response.read().decode("utf-8")
 
    html = ""
    try:
        response = urllib.request.urlopen(request)
        html = response.read().decode("utf-8")
    except urllib.error.URLError as e:
        if hasattr(e, "code"):
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
    return html
 
# 从 html 页面爬取数据
def getData(html):
    print("—————————— Process ——————————")
    datalist = []           # 用来存储爬取的网页信息
    # 1.逐一解析数据
    soup = BeautifulSoup(html, "html.parser")           # BeautifulSoup解析页面信息
    soup.prettify()
    for item in soup.find_all('tr', class_="item"):     # 查找符合要求的字符串
        data = []           # 保存一部电影所有信息
        # 2.生成一条记录
        data.append(item.td.a["title"])
        data.append(item.td.a["href"])
        data.append(item.td.img["src"])
        data.append(item.find('span', class_='rating_nums').text)
        data.append(item.find('span', class_='pl').text)
        data.append(item.p.text)
        # 3. 存入list
        datalist.append(data)
    return datalist
 
# 保存数据到表格
def saveData(datalist, savepath):
    print("—————————— save ——————————")
    book = xlwt.Workbook(encoding="utf-8",style_compression=0)   # 创建workbook对象
    sheet = book.add_sheet('豆瓣新片榜', cell_overwrite_ok=True)   # 创建工作表
    col = ("影片中文名", "电影详情链接", "图片链接", "评分", "评价数", "电影概况")
    for i in range(0, len(col)):
        sheet.write(0, i, col[i])           # 列名
    for i in range(0, len(datalist)):
        # print("第%d条" %(i+1))             # 输出语句，用来测试
        data = datalist[i]
        for j in range(0, len(col)):
            sheet.write(i+1, j, data[j])    # 数据
    if os.path.exists(savepath):
        os.remove(savepath)
    book.save(savepath)                     # 保存
    pass
 
if __name__ == "__main__":
    print("—————————— 开始执行 ——————————")
    # 1. 读取uri
    html = readHtml()
    # 2. 处理Html数据
    data = getData(html)
    # 3. 保存数据
    saveData(data, "豆瓣新片榜.xls")
    print("—————————— 爬取完毕 ——————————")

ModuleNotFoundError: No module named 'xlwt'